# **tf.data API**

> tf.data API를 사용하여 모델 성능 향상시키기

https://www.tensorflow.org/guide/data_performance?hl=ko



In [1]:
import tensorflow as tf

import time

**데이터 셋**

In [2]:
#Dataset클래스를 상속하여 ArtificialDataset을 제작
class ArtificialDataset(tf.data.Dataset):
  #iterator를 생성하는 함수를 generator라고 하며, yield키워드를 지니는 함수의 총칭이다.
  #yield는 iterator의 각 항목을 순서대로 생성하며, generator에 반드시 하나는 있어야 한다.
  def _generator(num_samples):
    # 파일 열기
    time.sleep(0.03)

    for sample_idx in range(num_samples):
      # 파일에서 데이터(줄, 기록) 읽기
      time.sleep(0.015)

      yield (sample_idx,)

  def __new__(cls, num_samples=3):
    #Dataset의 from_generator메서드는 설정된 generator를 받아 generator가 생성하는대로 Dataset을 생성한다.
    return tf.data.Dataset.from_generator(
      cls._generator,
      output_types=tf.dtypes.int64,
      output_shapes=(1,),
      args=(num_samples,)
    )

Generator이해하기

In [3]:
def number_generator():
    yield 0
    yield 1
    yield 2
#number_generator함수의 결괏값은 iterator이므로 for in 구문에 들어갈 수 있다.
for i in number_generator():
  print(i)
  #0,1,2순서대로 출력된 것을 볼 수 있다.

0
1
2


In [4]:
def number_generator(num):
  for i in range(0,num):
    yield(i)
for i in number_generator(7):
  print(i)
  #(0,7)범위의 정수가 순서대로 출력된것을 볼 수 있다.

0
1
2
3
4
5
6


**훈련루프**


> 데이터셋을 반복하는데 걸리는 시간을 측정하는 더미 훈련루프 정의



In [5]:
def benchmark(dataset, num_epochs=2):
  #perf_counter메서드는 time.sleep메서드로 일시정지한 시간을 포함해 측정한다.
  start_time = time.perf_counter()
  for epoch_num in range(num_epochs):
    #데이터 세트를 순회
    for sample in dataset:
      # 훈련 스텝마다 실행
      time.sleep(0.01)
  tf.print("실행 시간:", time.perf_counter() - start_time)

In [6]:
benchmark(ArtificialDataset())

실행 시간: 0.4265142440000034


**prefetch**

> 위의 코드는 ArtificialDataset()을 그대로 넣어 실행시간은 데이터를 읽고 쓰는 시간까지 포함한 시간이다. 이를 방지하기위해 텐서플로우 데이터세트는 prefetch기능으로 데이터를 미리 읽어놓을 수 있다.



In [7]:
benchmark(
    ArtificialDataset()
    .prefetch(tf.data.experimental.AUTOTUNE)
)
#실행시간이 절반으로 준 것을 볼 수 있다.

실행 시간: 0.21717367200000126


**데이터 추출 병렬화**


> 두개의 데이터 셋에서 동시에 데이터를 읽어오는 방법



순차 인터리브

In [8]:
#interleave는 두개의 데이터 셋에서 데이터를 순차적으로 인터리브한다.
#인터리브:하드웨어의 성능을 높이기 위해 두 데이터를 겹치지 않게 배치하는 데이터 처리 기법
benchmark(
    tf.data.Dataset.range(2)
    .interleave(ArtificialDataset)
)

Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead
실행 시간: 0.2375453870000115


In [9]:
#num_parallel_calls매개변수는 interleave메서드의 병렬처리 수준을 설정한다.
benchmark(
    tf.data.Dataset.range(2)
    .interleave(
        ArtificialDataset,
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
)
#실행시간이 줄어든 것을 볼 수 있다.

실행 시간: 0.18452532899999596


**데이터 변환 병렬화**

In [10]:
def mapped_function(s):
    # Do some hard pre-processing
    #py_function메서드는 주어진 파이썬 함수를 텐서플로우 함수로 변환한다.
    #py_function(파이썬함수,인자,반환자료형)
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s

In [11]:
benchmark(
    ArtificialDataset()
    .map(mapped_function)
)

실행 시간: 0.4424666959999968


In [12]:
#map메서드에서 num_parallel_calls매개변수를 통해 데이터세트의 각 샘플에 대한 매핑을 병렬화할 수 있다.
benchmark(
    ArtificialDataset()
    .map(
        mapped_function,
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
)

실행 시간: 0.309227342000014


**Cache**


> 캐시는 데이터셋의 작업 등을 메모리나 로컬 저장소에 저장할 수 있습니다. 원격 저장소에서 데이터를 읽어오는 경우 읽는 시간을 줄이기 위해서 로컬저장소 등에 저장하는 것이 일반적입니다.



In [13]:
benchmark(
    ArtificialDataset()
    .map(mapped_function),5
)

실행 시간: 1.1072506850000252


In [14]:
#epoch를 5번 적용
benchmark(
    #매핑 이후 캐시를 저장했기 때문에, 매핑 작업후 상태가 메모리에 저장되어 2번째 epoch부터는 이를 메모리에서 읽어오기만 한다.
    ArtificialDataset()
    .map(  # 캐시 전 시간이 많이 걸리는 작업 적용
        mapped_function
    ).cache(),5
)
#학습 시간이 눈에 띄게 줄어든 것을 볼 수 있다.

실행 시간: 0.3762319479999974


**매핑 벡터화**

> 사용자 정의 함수를 벡터화(즉, 한 번에 여러 입력에 대해 작동하도록)하는 기법



In [15]:
#앞서 만든 샘플 세개짜리 ArtificalDataset은 샘플 개수가 너무 적어 효과를 보기 어려우므로 샘플 10000개짜리 데이터셋을 만들어준다.
fast_dataset = tf.data.Dataset.range(10000)

def fast_benchmark(dataset, num_epochs=2):
  start_time = time.perf_counter()
  for _ in tf.data.Dataset.range(num_epochs):
    for _ in dataset:
      #더 빠른 실행 시간을 위해 time.sleep(0.01)구문을 뺐다.
      pass
  tf.print("실행 시간:", time.perf_counter() - start_time)

def increment(x):
    return x+1

In [16]:
#일반적인 스칼라 매핑을 사용했을때.
fast_benchmark(
    fast_dataset
    # 한 번에 한 항목씩 함수 적용
    .map(increment)
    # 배치
    .batch(256)
)

실행 시간: 1.144028654000067


In [17]:
#매핑 벡터화
#batch에 매핑을 적용한다.
fast_benchmark(
    fast_dataset
    .batch(256)
    # items의 배치에 함수 적용
    # tf.Tensor.__add__ 메서드는 이미 배치를 다룸
    .map(increment)
)
#시간이 절반 이하로 확 줄어든 것을 볼 수 있다.

실행 시간: 0.05037968700003148


**메모리 사용량 줄이기**

부분 계산 캐싱


> 매핑 이후 캐싱을 하면 사용량을 줄일 수 있다.



In [21]:
fast_dataset.map(increment).cache().map(increment)

<MapDataset shapes: (), types: tf.int64>